In [1]:
!pip install -q transformers einops accelerate langchain bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 732.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.9/273.9 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [4]:
model="meta-llama/Llama-2-7b-chat-hf"

In [5]:
tokenizer=AutoTokenizer.from_pretrained(model,use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:720: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [6]:
model=AutoModelForCausalLM.from_pretrained(model,
                                           device_map='auto',
                                           torch_dtype=torch.float16,
                                           use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:466: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [7]:
pipeline=transformers.pipeline("text-generation",
                               model=model,
                               tokenizer=tokenizer,
                               torch_dtype=torch.bfloat16,
                               device_map="auto",
                               # Maximum number of tokens to generate, a word is generally 2-3 tokens (minimum: 1)
                               max_new_tokens=512,
                               #Minimum number of tokens to generate. To disable this set it to -1. A word is generally 2-3 tokens (minimum 1)
                               min_new_tokens=-1,
                               #Adjusts the randomness of output
                               temperature=0.75,
                               do_sample=True,
                               top_k=30,
                               num_return_sequences=1,
                               eos_token_id=tokenizer.eos_token_id)

In [8]:
llm=HuggingFacePipeline(pipeline=pipeline, model_kwargs={'temperature':0})

In [9]:
llm.predict("Please provide a concise summary of the Book Harry Potter")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


"Please provide a concise summary of the Book Harry Potter and the Philosopher's Stone by J.K. Rowling.\nHarry Potter is an orphaned boy who discovers he is a wizard on his eleventh birthday. He attends Hogwarts School of Witchcraft and Wizardry, where he becomes entangled in a mystery involving the powerful Sorcerer's Stone. Along with his friends Ron Weasley and Hermione Granger, Harry must prevent the Stone from falling into the wrong hands, as it has the power to grant eternal life.\nHarry Potter and the Philosopher's Stone is a fantasy novel that follows the journey of a young boy as he discovers his true identity and battles against the forces of evil. The book is the first in the Harry Potter series and introduces readers to the magical world of Hogwarts and its inhabitants."

In [10]:
B_INST, E_INST= "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<<SYS>>\n\n"

In [11]:
DEFAULT_SYSTEM_PROMPT="""\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
"""

In [28]:
instruction = "Convert the following text from English to Hindi: \n\n{text}"

In [29]:
SYSTEM_PROMPT=B_SYS + DEFAULT_SYSTEM_PROMPT + E_SYS

In [30]:
SYSTEM_PROMPT

"<<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n\n<<SYS>>\n\n"

In [31]:
template = B_INST + SYSTEM_PROMPT + instruction + E_INST

In [32]:
print(template)

[INST]<<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.

<<SYS>>

Convert the following text from English to Hindi: 

{text}[/INST]


In [33]:
prompt=PromptTemplate(input_variables=["text"], template=template)

In [34]:
text = "How are you?"

In [35]:
LLM_Chain=LLMChain(llm=llm, prompt=prompt)

In [36]:
LLM_Chain.run(text)

'[INST]<<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don\'t know the answer to a question, please don\'t share false information.\n\n<<SYS>>\n\nConvert the following text from English to Hindi: \n\nHow are you?[/INST]  I apologize, but I cannot provide a direct translation of "How are you?" from English to Hindi as it is a common English greeting that may not have an exact equivalent in Hindi. However, here are a few ways you could express a similar sentiment in Hindi:\n\n* "कैसे हैं आप?" (Kaise hain aap?) - This is a common way to ask someone how they are doing in Hindi.\n* "आपका कैसा है?" (Aap

Now this is giving large response since we are using Default System prompt. Now we need to customize the prompt to get the exact answer that we are looking for.

### Custom System Prompt

In [43]:
instruction = "Convert the following text from English to Hindi: \n\n{text}"
CUSTOM_SYSTEM_PROMPT="You are an advanced assistant that excels at translation. Just give the output translation nothing else."
SYSTEM_PROMPT=B_SYS+CUSTOM_SYSTEM_PROMPT+E_SYS
template=B_INST+SYSTEM_PROMPT+instruction+E_INST

In [44]:
print(template)

[INST]<<SYS>>
You are an advanced assistant that excels at translation. Just give the output translation nothing else.
<<SYS>>

Convert the following text from English to Hindi: 

{text}[/INST]


In [45]:
prompt=PromptTemplate(input_variables=["text"], template=template)

In [46]:
text = "How are you?"
LLM_Chain=LLMChain(llm=llm, prompt=prompt)
print(LLM_Chain.run(text))

[INST]<<SYS>>
You are an advanced assistant that excels at translation. Just give the output translation nothing else.
<<SYS>>

Convert the following text from English to Hindi: 

How are you?[/INST]  Here is the translation of "How are you?" from English to Hindi:

कैसे हैं आप?
